In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import re
import time

Importing the libraries.

Reading the data from the csv file.

In [39]:
df=pd.read_csv("MgB.csv")

Head function usually gives the top rows.

In [41]:
df.head(3)

,Title,Furnishing_status,Agent,OperatingTime,SecurityDeposit,Premium,Area
0,1 BHK Builder Floor for Rent in NEB Valley Soc...,FurnishingSemi-Furnished,Agent: Nagpal Associates,Operating Since: 2005,"₹10,000",NaN,NaN
1,Studio Apartment for Rent in Project Amar Colo...,FurnishingSemi-Furnished,Agent: Bhalla Properties,Operating Since: 2000,"₹10,000",NaN,NaN
2,1 BHK Builder Floor for Rent in Saket New Delhi,FurnishingSemi-Furnished,Agent: S S Deswal,NaN,"₹10,000",NaN,NaN


In the below cell ,we are extracting the particular string from the title column to form a new column called area in the dataframe.
In re.search(pattern,string) we usually searches the pattern in that particular string.Here also we're doing the same but using group(1) to capture the string only inside the parenthesis.
so now coming to the code,there we can see that we used search pattern twice.so the first search pattern is using ifelse condition to confirm whether there is a match or not.If there is no match it will return nan value.If there is a match ,it will go to second second search pattern and it will extract the match.so,why second search pattern because we cannot extract the match directly from the search pattern itself because if there is no match it will raise an error if we extract the match directly.
Now we are applying this entire expression to the anonymous function called lambda.and after that in we use a method called apply in which allows us to apply function to either row or column.
we are doing the same procedure to the all the columns to which we are extracting the data from the title column.



In [43]:
df['Area'] = df['Title'].apply(lambda title: re.search(r'in (.*?) New Delhi', title).group(1) if re.search(r'in (.*?) New Delhi', title) else np.NaN)


In [45]:
df['BHK'] = df['Title'].apply(lambda title: re.search(r'\d+\s*BHK', title).group() if re.search(r'\d+\s*BHK', title) else np.NaN)


In [47]:
df['BHK'] = df['BHK'].str.replace(r' BHK', '', regex=True)


In the below cell we are extracting the data directly using string method.

In [49]:
df['Furnishing_status'] = df['Furnishing_status'].str.extract(r'(Unfurnished|Semi-Furnished|Furnished)')


In the below cell we are replace the values in the search pattern with space and after that using strip we are getting string that needed without any unwanted spaces.

In [51]:
df['Owner'] = df['Agent'].str.replace(r'^(Agent:|Owner:|:|Property|Mansion|Properties|Builders|Associates)\s*', '', regex=True).str.strip()


Converting the Security deposit column into the numeric column and applying the pattern to get the desired value.
In premium column using where condition we are setting the value whether it is premium or not.

In [53]:

df['SecurityDeposit'] = pd.to_numeric(df['SecurityDeposit'].replace(r'[₹,]', '', regex=True), errors='coerce')

df['Premium'] = np.where(df['SecurityDeposit'] > 5000, 'Premium', 'Non-Premium')



In [ ]:
In the below column we are converting it to string to extract the data after that we are converting again into int.

In [55]:
df['OperatingTime'] = df['OperatingTime'].astype(str)  
df['OperatingTime'] = df['OperatingTime'].str.extract(r'(\d{4})')  
df['OperatingTime'] = df['OperatingTime'].astype('Int64')  


In [57]:
df['Residence'] = df['Title'].str.extract(r'(House|Apartment)', flags=re.IGNORECASE, expand=False).fillna('Other')


In [59]:
df.rename(columns={"Residence":"ResidencyType"},inplace=True)

In [61]:
df.drop(columns="Agent",inplace=True)

In [63]:
df.fillna({col: 0 for col in df.select_dtypes(include=['number']).columns}, inplace=True)
df.fillna({col: 'Unknown' for col in df.select_dtypes(include=['object']).columns}, inplace=True)

In [65]:
df.head(2)

,Title,Furnishing_status,OperatingTime,SecurityDeposit,Premium,Area,BHK,Owner,ResidencyType
0,1 BHK Builder Floor for Rent in NEB Valley Soc...,Semi-Furnished,2005,10000,Premium,"NEB Valley Society, Saket",1,Nagpal Associates,Other
1,Studio Apartment for Rent in Project Amar Colo...,Semi-Furnished,2000,10000,Premium,"Project Amar Colony, Lajpat Nagar 4",Unknown,Bhalla Properties,Apartment


In [67]:
df.drop(columns="Title",inplace=True)

In [69]:
df.to_csv("MGB.csv")